The following key measures are provided:

<ul id="docNav">

<li> <a href="#systolic_bp">Blood Pressure Monitoring</a>
<li> <a href="#qrisk2">Cardiovascular Disease 10 Year Risk Assessment</a>
<li> <a href="#cholesterol">Cholesterol Testing</a>
<li> <a href="#ALT">Liver Function Testing - Alanine Transferaminase (ALT)</a>
<li> <a href="#serum_tsh">Thyroid Testing</a>
<li> <a href="#rbc_fbc">Full Blood Count - Red Blood Cell (RBC) Testing</a>
<li> <a href="#hba1c">Glycated Haemoglobin A1c Level (HbA1c)</a>
<li> <a href="#serum_sodium">Renal Function Assessment - Sodium Testing</a>
<li> <a href="#asthma">Asthma Reviews</a>
<li> <a href="#copd">Chronic Obstrutive Pulmonary Disease (COPD) Reviews</a>
<li> <a href="#med_review">Medication Review</a>

</ul>

<h3 class="details">Summary of results</h3>

These key measures demonstrated substantial changes in clinical activity throughout the COVID-19 pandemic. Using our [classification of activity change methods](https://reports.opensafely.org/reports/sro-measures/), six of the measures recovered to their pre-pandemic baseline within a year of the pandemic, showing a rapid, adaptive response by primary care in the midst of a global health pandemic. The remaining five measures showed a more sustained drop in activity; asthma and COPD reviews did not recover to their pre-pandemic baseline until around August 2021 and blood pressure monitoring, cardiovascular disease risk assessment and medication reviews had a sustained drop in activity that persisted up to December 2021. Since December 2021 activity rates for all measures have recovered and in some cases are now above the pre-pandemic baseline.

<h3 class="details">Findings in context</h3>

Discussion of the specific causes and reasons for the changes in narrow measures of clinical activity we have described is best addressed through quantitative analyses that identify practices in high and low deciles to approach for targeted qualitative interviews with patients and front line staff. However we believe the following broad points may help aid interpretation. Our measures reflect only a few areas of high volume clinical activity; decreases may reflect appropriate prioritisation of other clinical activity. For example NHS Health Checks, which are used to detect early signs of high blood pressure, heart disease or type 2 diabetes, were paused during the pandemic; this is likely to explain the sustained drop in activity in cardiovascular disease risk assessment and blood pressure monitoring. However, in specific cases this may reflect changes in the style of delivery of a clinical activity, rather than the volume: for example, where patients record their own blood pressure at home since, as we have previously highlighted, home monitoring of blood pressure may not be recorded completely or consistently in GP records. In addition, not all reductions should be interpreted as problematic: as part of the COVID-19 recovery, health systems are aiming to be more resilient, responsive and sustainable; complete recovery may not always be appropriate and reductions in clinical activity across some domains may reflect rational reprioritisation of activity. Where these changes in priority have not been nationally planned, data analyses such as ours may help to rapidly identify the pragmatic changes in prioritisation being made by individual dispersed organisations or people across the healthcare ecosystem before those changes are explicitly surfaced or discussed through other mechanisms. For more detail, please see our published manuscript [here](https://doi.org/10.7554/eLife.84673).

In [ ]:
import pandas as pd
import numpy as np

from pandas.plotting import register_matplotlib_converters

from utilities import get_number_practices, get_percentage_practices, deciles_chart, OUTPUT_DIR

from IPython.display import HTML, display, Markdown
import matplotlib.pyplot as plt
register_matplotlib_converters()

%matplotlib inline
%config InlineBackend.figure_format='png'
plt.rcParams["axes.grid"] = True

In [ ]:
measures_df = pd.read_csv('../output/measures.csv', parse_dates=["interval_start", "interval_end"])

In [ ]:
%%capture --no-display

sentinel_measures = ["qrisk2", "asthma", "copd", "sodium", "cholesterol", "alt", "tsh", "rbc", 'hba1c', 'systolic_bp', 'medication_review']

sentinel_measure_codelist_mapping_dict = {"systolic_bp":"opensafely-systolic-blood-pressure-qof", "qrisk":"opensafely-cvd-risk-assessment-score-qof", "cholesterol": "opensafely-cholesterol-tests", "alt": "opensafely-alanine-aminotransferase-alt-tests", "tsh": "opensafely-thyroid-stimulating-hormone-tsh-testing", "rbc": "opensafely-red-blood-cell-rbc-tests", "hba1c": "opensafely-glycated-haemoglobin-hba1c-tests", "sodium": "opensafely-sodium-tests-numerical-value", "asthma": "opensafely-asthma-annual-review-qof", "copd": "opensafely-chronic-obstructive-pulmonary-disease-copd-review-qof"}

codelist_dict = {}
for measure in sentinel_measures:

    if measure =="qrisk2":
        measure= "qrisk"
    
    if measure == 'medication_review':
        codelist_1 = pd.read_csv('../codelists/opensafely-care-planning-medication-review-simple-reference-set-nhs-digital.csv')
        codelist_1['term'] = codelist_1['term'].str.rstrip('(procedure)').str.rstrip()
        codelist_2 = pd.read_csv('../codelists/nhsd-primary-care-domain-refsets-medrvw_cod.csv')
        codelist_dict[measure] = codelist_1.merge(codelist_2, on=['code', 'term'], how='outer')
    else:
        codelist_name = sentinel_measure_codelist_mapping_dict[measure]
        codelist = pd.read_csv(f'../codelists/{codelist_name}.csv')
        codelist_dict[measure] = codelist

In [ ]:
def drop_irrelevant_practices(df):
    """Drops irrelevant practices from the given measure table.

    An irrelevant practice has zero events during the study period.

    Args:
        df: A measure table.

    Returns:
        A copy of the given measure table with irrelevant practices dropped.
    """

    is_relevant = df.groupby("practice").ratio.any()
    return df[df.practice.isin(is_relevant[is_relevant == True].index)]

def create_child_table(df, code_df, code_column, term_column, measure, nrows=5):
    """
    Args:
        df: A measure table.
        code_df: A codelist table.
        code_column: The name of the code column in the codelist table.
        term_column: The name of the term column in the codelist table.
        measure: The measure ID.
        nrows: The number of rows to display.

    Returns:
        A table of the top `nrows` codes.
    """
    event_counts = (
        df.groupby(f"{measure}_code")["numerator"]
        .sum()  # We can't use .count() because the measure column contains zeros.
        .rename_axis(code_column)
        .rename("Events")
        .reset_index()
        .sort_values("Events", ascending=False)
    )

    # round events to nearest 5
    event_counts["Events"] = np.round(event_counts["Events"] / 5) * 5

    # calculate % makeup of each code
    total_events = event_counts["Events"].sum()
    event_counts["Proportion of codes (%)"] = round(
        (event_counts["Events"] / total_events) * 100, 2
    )

    # Gets the human-friendly description of the code for the given row
    # e.g. "Systolic blood pressure".
    code_df = code_df.set_index(code_column).rename(
        columns={term_column: "Description"}
    )
    event_counts = event_counts.set_index(code_column).join(code_df).reset_index()

    # Cast the code to an integer.
    event_counts[code_column] = event_counts[code_column].astype(int)

    # check that codes not in the top 5 rows have >5 events
    outside_top_5_percent = 1 - ((event_counts.head(5)["Events"].sum()) / total_events)

    if 0 < (outside_top_5_percent * total_events) <= 5:
        # drop percent column
        event_counts = event_counts.loc[:, ["code", "Description"]]

    else:
        # give more logical column ordering
        event_counts_with_count = event_counts.loc[
            :, ["code", "Description", "Events", "Proportion of codes (%)"]
        ]

        event_counts = event_counts.loc[
            :, ["code", "Description", "Proportion of codes (%)"]
        ]

    if len(event_counts["code"]) > 1:
        event_counts.loc[
            event_counts["Proportion of codes (%)"] == 0, "Proportion of codes (%)"
        ] = "< 0.005"
        event_counts.loc[
            event_counts["Proportion of codes (%)"] == 100, "Proportion of codes (%)"
        ] = "> 99.995"

        event_counts_with_count.loc[
            event_counts_with_count["Proportion of codes (%)"] == 0,
            "Proportion of codes (%)",
        ] = "< 0.005"
        event_counts_with_count.loc[
            event_counts_with_count["Proportion of codes (%)"] == 100,
            "Proportion of codes (%)",
        ] = "> 99.995"

    # return top n rows
    return event_counts.head(5), event_counts_with_count.head()


def get_number_events_mil(measure_table):
    """Gets the number of events per million, rounded to 2DP.

    Args:
        measure_table: A measure table.
        measure_id: The measure ID.
    """
    num_events = measure_table["numerator"].sum()
    return num_events, np.round(num_events / 1_000_000, 2)

In [ ]:
measure_dfs = {}
child_tables = {}

for measure in sentinel_measures:
    if measure =="qrisk2":
        measure = "qrisk"
    
    measure_subset_practice = measures_df.loc[measures_df['measure'] == f"{measure}_practice", ["measure", "interval_start", "interval_end", "ratio", "numerator", "denominator", "practice"]]
    measure_subset_code = measures_df.loc[measures_df['measure'] == f"{measure}_code", ["measure", "interval_start", "interval_end", "ratio", "numerator", "denominator", f"{measure}_code"]]
    

    measure_subset_practice = drop_irrelevant_practices(measure_subset_practice)

    measure_dfs[measure] = {"practice": measure_subset_practice, "code": measure_subset_code}



    event_counts, event_count_with_count = create_child_table(measure_subset_code, codelist_dict[measure], 'code', 'term', measure)
    child_tables[measure] = event_counts, event_count_with_count

In [ ]:
def generate_sentinel_measure(
        measure_df,
        measure,
        code_column,
        codelist_links,
    ):
    """Generates tables and charts for the measure with the given ID.

    Args:
        data_dict: A mapping of measure IDs to measure tables.
        data_dict_practice: A mapping of measure IDs to "practice only" measure tables.
        codelist_dict: A mapping of measure IDs to codelist tables.
        measure: A measure ID.
        code_column: The name of the code column in the codelist table.
        term_column: The name of the term column in the codelist table.
        dates_list: Not used.
        interactive: Flag indicating whether or not the chart should be interactive.
    """

    practices_included = get_number_practices(measure_df)
    num_events, num_events_mil = get_number_events_mil(measure_df)
    measure_df["rate_per_1000"] = measure_df["ratio"] * 1000
    deciles_chart(
        measure_df,
        "interval_start",
        "rate_per_1000",
        interactive=False,
        height=600,
        width=1000,
        output_path=f"{OUTPUT_DIR}/deciles_chart_{measure}.png",
    )

    childs_df, childs_df_with_count = child_tables[measure]

  
    display(
        Markdown(f"Practices included: {practices_included}"),
    )

    childs_df = childs_df.rename(columns={code_column: code_column.title()})
    childs_df.to_csv(f"{OUTPUT_DIR}/code_table_{measure}.csv")

    childs_df_with_count = childs_df_with_count.rename(
        columns={code_column: code_column.title()}
    )
    childs_df_with_count.to_csv(f"{OUTPUT_DIR}/code_table_{measure}_with_count.csv")

    if len(codelist_links)>1:
        display(
            Markdown(f"#### Most Common Codes <a href={codelist_links[0]}>(Codelist 1)</a>, <a href={codelist_links[1]}>(Codelist 2)</a>"),
            HTML(childs_df.to_html(index=False)),
        )

    else:
        display(
            Markdown(f"#### Most Common Codes <a href={codelist_links[0]}>(Codelist)</a>"),
            HTML(childs_df.to_html(index=False)),
        )

    display(
        Markdown(f"Total events: {num_events_mil:.2f}M"),
    )

    return

<a id="systolic_bp"></a>
## Blood Pressure Monitoring

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/systolic-blood-pressure-qof/3572b5fb/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

A commonly-used assessment used to identify patients with hypertension or to ensure optimal treatment for those with known hypertension.  This helps ensure appropriate treatment, with the aim of reducing long term risks of complications from hypertension such as stroke, myocardial infarction and kidney disease. 

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic. 

In [ ]:
generate_sentinel_measure(
    measure_dfs['systolic_bp']['practice'],
    'systolic_bp',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/systolic-blood-pressure-qof/3572b5fb/"]
)

<a id="qrisk2"></a>
## Cardiovascular Disease 10 year Risk Assessment

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/cvd-risk-assessment-score-qof/1adf44a5/">this codelist</a>.

<h3 class="details">What is it and why does it matter? </h3>

A commonly-used risk assessment used to identify patients with an increased risk of cardiovascular events in the next 10 years. This helps ensure appropriate treatment, with the aim of reducing long term risks of complications such as stroke or myocardial infarction. 

In [ ]:
generate_sentinel_measure(
    measure_dfs['qrisk']['practice'],
    'qrisk',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/cvd-risk-assessment-score-qof/1adf44a5/"]
)

<a id="cholesterol"></a>
## Cholesterol Testing

The codes used this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/cholesterol-tests/09896c09/">Codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

A commonly-used blood test used as part of a routine cardiovascular disease 10 year risk assessment and also to identify patients with lipid disorders (e.g. familial hypercholesterolaemia). This helps ensure appropriate treatment, with the aim of reducing long term risks of complications such as stroke or myocardial infarction.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure(
    measure_dfs['cholesterol']['practice'],
    'cholesterol',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/cholesterol-tests/09896c09/"]
)

<a id="ALT"></a>
## Liver Function Testing - Alanine Transferaminase (ALT)

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/alanine-aminotransferase-alt-tests/2298df3e/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

An ALT blood test is one of a group of liver function tests (LFTs) which are used to detect problems with the function of the liver.  It is often used to monitor patients on medications which may affect the liver or which rely on the liver to break them down within the body. They are also tested for patients with known or suspected liver dysfunction.  

<h3 class="details">Caveats</h3>

**In a small number of places, an ALT test may NOT be included within a liver function test**. We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure(
    measure_dfs['alt']['practice'],
    'alt',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/alanine-aminotransferase-alt-tests/2298df3e/"]
)

<a id="serum_tsh"></a>
## Thyroid Testing

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/thyroid-stimulating-hormone-tsh-testing/11a1abeb/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

TSH is used for the diagnosis and monitoring of hypothyroidism and hyperthyroidism, including making changes to thyroid replacement therapy dosing.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure(
    measure_dfs['tsh']['practice'],
    'tsh',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/thyroid-stimulating-hormone-tsh-testing/11a1abeb/"]
)

<a id="rbc_fbc"></a>
## Full Blood Count - Red Blood Cell (RBC) Testing

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/red-blood-cell-rbc-tests/576a859e/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

RBC is completed as part of a group of tests referred to as a full blood count (FBC), used to detect a variety of disorders of the blood, such as anaemia and infection.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure(
    measure_dfs['rbc']['practice'],
    'rbc',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/red-blood-cell-rbc-tests/576a859e/"]
)

<a id="hba1c"></a>
## Glycated Haemoglobin A1c Level (HbA1c)

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/glycated-haemoglobin-hba1c-tests/62358576/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

HbA1c is a long term indicator of diabetes control. NICE guidelines recommend that individuals with diabetes have their HbA1c measured at least twice a year. Poor diabetic control can place individuals living with diabetes at an increased risk of the complications of diabetes.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure(
    measure_dfs['hba1c']['practice'],
    'hba1c',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/glycated-haemoglobin-hba1c-tests/62358576/"]
)

<a id="serum_sodium"></a>
## Renal Function Assessment - Sodium Testing

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/sodium-tests-numerical-value/32bff605/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

Sodium is completed as part of a group of tests referred to as a renal profile, used to detect a variety of disorders of the kidneys. A renal profile is also often used to monitor patients on medications which may affect the kidneys or which rely on the kidneys to remove them from the body.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure(
    measure_dfs['sodium']['practice'],
    'sodium',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/sodium-tests-numerical-value/32bff605/"]
)

<a id="asthma"></a>
## Asthma Reviews

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/asthma-annual-review-qof/33eeb7da/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

The British Thoracic Society and Scottish Intercollegiate Guidelines Network on the management of asthma recommend that people with asthma receive a review of their condition at least annually. If a patient has not been reviewed, it is possible that their asthma control may have worsened, leading to a greater chance of symptoms and admission to hospital.

In [ ]:
generate_sentinel_measure(
    measure_dfs['asthma']['practice'],
    'asthma',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/asthma-annual-review-qof/33eeb7da/"]
)

<a id="copd"></a>
## Chronic Obstructive Pulmonary Disease (COPD) Reviews

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/chronic-obstructive-pulmonary-disease-copd-review-qof/01cfd170/">this codelist</a>.  

<h3 class="details">What is it and why does it matter?</h3>

It is recommended by NICE that all individuals living with COPD have an annual review with the exception of individuals living with very severe (stage 4) COPD being reviewed at least twice a year.
If a patient has not been reviewed, it is possible that their COPD control may have worsened, leading to a greater chance of symptoms and admission to hospital.

In [ ]:
generate_sentinel_measure(
    measure_dfs['copd']['practice'],
    'copd',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/chronic-obstructive-pulmonary-disease-copd-review-qof/01cfd170/"]
)

<a id="med_review"></a>
## Medication Reviews

The codes used for this measure are a combination of codes available in <a href="https://www.opencodelists.org/codelist/opensafely/care-planning-medication-review-simple-reference-set-nhs-digital/61b13c39/"> this NHS Digital care planning medication review refset (Note refset now inactive, but codes within refset are frequently used and so continue to be included within this report)</a> and <a href="https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/medrvw_cod/20200812/">this primary care domain medication review refset</a>.
 
<h3 class="details">What is it and why does it matter?</h3>

Many medicines are used long-term and they should be reviewed regularly to ensure they are still safe, effective and appropriate.
Medication review is a broad term ranging from a notes-led review without a patient, to an in-depth Structured Medication Review with multiple appointments and follow-up. The codelist provided captures all types of reviews to give an overview of medication reviews in primary care.

In [ ]:
generate_sentinel_measure(
    measure_dfs['medication_review']['practice'],
    'medication_review',
    'code',
    codelist_links=["https://www.opencodelists.org/codelist/opensafely/care-planning-medication-review-simple-reference-set-nhs-digital/61b13c39/"]
)